In [0]:
# Assignment 1 

import os, struct
from array import array as pyarray
from numpy import  array, zeros
import numpy as np

def load_mnist(dataset, digits=np.arange(10), path="/home/manoj/Documents/Sem 3/CUDA_LabVision/Session 1/data/data/"):
    
    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols) )
    labels = zeros((N ) )
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels
    
train_images , train_labels = load_mnist("training")
test_images , test_labels = load_mnist("testing")


train_x = train_images.reshape(60000,784)
train_y = train_labels
test_x = test_images.reshape(10000,784)
test_y = test_labels

train_x /= 255.
test_x /= 255.
print train_x.shape, train_labels.shape, test_x.shape, test_labels.shape 

(60000, 784) (60000,) (10000, 784) (10000,)


In [0]:
# Softmax Activation(y = Wx+b)

def softmax(x):
    x -= np.max(x)
    sm = (np.exp(x).T / np.sum(np.exp(x))).T
    return np.argmax(sm)


In [0]:
# MSE

def MSE(y,batch_y):
    y = y.T
    error = (batch_y - y) 
    return error


In [0]:
# Training the network

def train_network(batch_x,batch_y, W): 
    for epoch in range(100):  
        inputs = batch_x
        # feedforward step 1
        XW = np.dot(inputs , W) + b
        # feedforward step 2
        y = softmax(XW)
        # backpropagation step 1
        error_computed = MSE(y,batch_y)
        W = update_weights(error_computed, inputs, W)
        return W

In [0]:
# Initialization of weights

import numpy as np
np.random.seed(1)

# Weight
W = np.random.rand(train_x.shape[1])

# Bias
b = np.zeros((1,1))

# Learning rate
learning_rate = 0.001
batch_size_each = 20
for k in range(1):
    for j in range(len(train_x)):
        if (j == 0):
            batch = train_x[:batch_size_each]
            batch_y = train_y[:batch_size_each]
        elif(j%20 == 0):
            batch = train_x[j-20:j]
            batch_y = train_y[j-20:j]
        W=train_network(batch,batch_y, W)

In [0]:
# Update Weights

def update_weights(error,batch_x, W):
    delta = np.dot(batch_x.T,error)
    W+= delta
    return W

In [0]:
# Test dataset prediction

actual_y = softmax(np.dot(test_x,W)+b)
np.equal(actual_y,test_y)
a,b=np.unique(actual_y,return_counts=True)
test_y.astype(float)
accurate_predictions = (len(test_y)-a[0])*100/len(test_y)
accurate_predictions

74